In [1]:
import os
import numpy as np
import json
import string
from random import randint

import tensorflow as tf
import tensorflow.contrib.seq2seq as seq2seq

from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper, ResidualWrapper

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.layers.core import Dense
from tensorflow.python.util import nest

from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.contrib.seq2seq.python.ops import beam_search_decoder

In [2]:
path = "../../data/MLDS_hw2_data/"

testdir = "testing_data/feat/"

testfiles = os.listdir(path+testdir)

testdata = {}
for i in range(0,len(testfiles)):
    testdata[str.split(testfiles[i],".")[0]+'.'+str.split(testfiles[i],".")[1]] = np.load(path+testdir+testfiles[i])
print(len(testdata))

testjsonfile = open(path+"testing_label.json","r")

testjson = json.load(testjsonfile)

encodeWords = np.load("encodeWords.npy").item()
decodeWords = np.load("decodeWords.npy").item()

print(len(encodeWords))
print(len(decodeWords))

max_seq_length = 21

def getStr(ints):
    sentence = ' '.join([decodeWords[int] for int in ints])
    sentence = sentence.replace('<BOS> ','').replace(' <EOS>', '')
    return sentence

def getTestDataSets():
    x_data = np.zeros((100,80,4096),dtype="float32")

    i = 0
    for x in testjson:
        name = x["id"]
        temp = testdata[name]
        x_data[i] = temp
        i=i+1
    return x_data

100
6061
6061


In [3]:
unit = 512
inputs = tf.placeholder(tf.float32,[None,80,4096])
batch_size = tf.shape(inputs)[0]
sequence_length = tf.fill([batch_size], max_seq_length)
beam_width = 3
start_tokens = tf.fill([batch_size*beam_width], encodeWords["<BOS>"])

def lstm_cell():
  return tf.contrib.rnn.BasicLSTMCell(unit)

encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)])

encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, inputs, dtype=tf.float32)

tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(encoder_outputs, beam_width)
tiled_encoder_state = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
batch_size_beam = tf.shape(tiled_encoder_outputs)[0]

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units=unit, memory=tiled_encoder_outputs)

decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)])

attention_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism)

initial_state = attention_cell.zero_state(dtype=tf.float32, batch_size=batch_size * beam_width)
initial_state = initial_state.clone(cell_state=tiled_encoder_state) 

embedding = tf.Variable(tf.random_uniform([len(encodeWords), unit], -0.1, 0.1, dtype=tf.float32))

output_projection_layer = Dense(len(encodeWords), use_bias=False)

#train
#helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding, start_tokens, encodeWords["<EOS>"])
decoder = tf.contrib.seq2seq.BeamSearchDecoder(attention_cell, embedding, start_tokens, encodeWords["<EOS>"], initial_state, beam_width, output_layer=output_projection_layer)
decoder_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, maximum_iterations=max_seq_length)

outputs = decoder_outputs.predicted_ids

print("I think it's OK!")

I think it's OK!


In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver.restore(sess, "model1000.ckpt")

# _,l,predict = sess.run([minimize, loss, sample], feed_dict={inputs: x_data})
# print(sample)

ResourceExhaustedError: OOM when allocating tensor with shape[4608,2048]
	 [[Node: rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Initializer/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, _class=["loc:@rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Initializer/random_uniform/shape)]]

Caused by op 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Initializer/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/l